In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import sleep
from datetime import datetime

# 데이터 불러오기

In [4]:
# 마스터테이블 과 14~21년도 매매데이터 불러오기
df = pd.read_csv('data/MasterTable/MTF_비인도데프_20211108180730.csv')
df_1421 = pd.read_csv('data/MasterMotherTable/df_mae_1421_11월04일17시29분.csv')

df_1421

,시군구,번지,본번,부번,단지명,전용면적_미제,계약년월,계약일,거래금액,층,건축년도,지역,전용면적_평,지번주소,new_Date,quater,자치구_동,id
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),78,201401,29,550.0,7,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2014-01-29,2014_1Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),78,201411,10,595.0,4,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2014-11-10,2014_4Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,80,201404,3,670.0,4,1987,서울,24.0,서울특별시 강남구 개포동 658-1,2014-04-03,2014_2Q,개포동,서울특별시 강남구 개포동_개포6차우성아파트1동~8동_80
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,68,201407,24,588.0,4,1987,서울,20.0,서울특별시 강남구 개포동 658-1,2014-07-24,2014_3Q,개포동,서울특별시 강남구 개포동_개포6차우성아파트1동~8동_68
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,55,201408,29,500.0,2,1987,서울,17.0,서울특별시 강남구 개포동 658-1,2014-08-29,2014_3Q,개포동,서울특별시 강남구 개포동_개포6차우성아파트1동~8동_55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075293,인천광역시 중구 항동7가,91-2,91,2,연안아파트,54,202105,24,240.0,5,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-05-24,2021_2Q,항동7가,인천광역시 중구 항동7가_연안아파트_54
2075294,인천광역시 중구 항동7가,91-2,91,2,연안아파트,54,202106,14,240.0,3,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-06-14,2021_2Q,항동7가,인천광역시 중구 항동7가_연안아파트_54
2075295,인천광역시 중구 항동7가,91-2,91,2,연안아파트,54,202106,19,260.0,5,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-06-19,2021_2Q,항동7가,인천광역시 중구 항동7가_연안아파트_54
2075296,인천광역시 중구 항동7가,91-2,91,2,연안아파트,54,202106,29,265.0,5,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-06-29,2021_2Q,항동7가,인천광역시 중구 항동7가_연안아파트_54


In [7]:
df_merge = pd.merge(df,df_1421.loc[df_1421['id'].drop_duplicates().index,['id','지번주소']],how='left',on='id')
id_list = df_merge['id'].drop_duplicates(keep='first')

In [13]:
df2 = df3.loc[id_list.index][['id','지번주소']]

# 리인덱싱
df2.reset_index(drop=True,inplace=True)

In [19]:
# id를 900개씩 끊어서 저장. (API에 모든 데이터를 돌릴 수 없었음)
for i in range(1,24):
    globals()['id_list{}'.format(i)] = df2.iloc[int(900*(i-1)):int(900*i),:]
id_list24 = df2.iloc[int(900*i):]

# API: 위도, 경도 수집

In [20]:
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [21]:
client_id = 'fsid7bs1f2'
client_pw = 'RXQI9DaURusBXb10v6zgkeifAJXl6t2iOTxjSGaR'

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

for i in range(1,25):
    print(i)
    sleep(10)
    lat = []  #위도
    lng = []  #경도
    for add in globals()['id_list{}'.format(i)]['지번주소']:
        add_urlenc = parse.quote(add)
        url = api_url + add_urlenc
        request = Request(url)
        request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
        request.add_header('X-NCP-APIGW-API-KEY', client_pw)
        try:
            response = urlopen(request)
        except:
            pass
        try:
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read().decode('utf-8')
                response_body = json.loads(response_body)
                if 'addresses' in response_body:
                    lat.append(response_body['addresses'][0]['y'])
                    lng.append(response_body['addresses'][0]['x'])
                else:
                    lat.append('')
                    lng.append('')
                    print(add)
        except:
            print('Response error code : %d' % rescode)
            lat.append('')
            lng.append('')
    globals()['df_location{}'.format(i)] = pd.DataFrame({'id':globals()['id_list{}'.format(i)]['id'],'위도':lat, '경도':lng})

1
2
3
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
4
5
6
7
Response error code : 200
Response error code : 200
Response error code : 200
8
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
9
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
10
Response error code : 200
11
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error code : 200
Response error

In [22]:
df_list = []
for i in range(1,25):
    df_list.append(globals()['df_location{}'.format(i)])
df_location = pd.concat(df_list,axis=0)

### 데이터 77개가 데이터를 불러오지 못함. 

In [24]:
df_temp = df_location[df_location['위도']== ''].copy()
df_temp

,id,위도,경도
1807,경기도 김포시 걸포동_한강메트로자이2단지_60,,
1808,경기도 김포시 걸포동_한강메트로자이2단지_75,,
1809,경기도 김포시 걸포동_한강메트로자이2단지_85,,
2120,경기도 남양주시 다산동_다산 펜테리움 리버테라스 Ⅰ_85,,
5538,경기도 시흥시 배곧동_시흥배곧신도시 이지더원2차 에듀그린_85,,
...,...,...,...
20611,인천광역시 연수구 동춘동_송도파크자이_77,,
20612,인천광역시 연수구 동춘동_송도파크자이_85,,
20617,인천광역시 연수구 동춘동_연수 서해그랑블 1단지_71,,
20618,인천광역시 연수구 동춘동_연수 서해그랑블 1단지_85,,


In [26]:
# 77개 아이디 수정
list_address = ['경기도 김포시 걸포동 180','경기도 김포시 걸포동 180','경기도 김포시 걸포동 180','경기도 남양주시 다산동 6215','경기도 시흥시 배곧동 279','경기도 시흥시 장현동 72-1',
               '경기도 시흥시 장현동 369','경기도 용인시 기흥구 구갈동 227-32','경기도 용인시 기흥구 구갈동 234-4','경기도 용인시 기흥구 구갈동 46-11','경기도 용인시 기흥구 구갈동 46-11',
               '경기도 용인시 기흥구 구갈동 46-11','경기도 용인시 수지구 고기로45번길 40-18','경기도 용인시 수지구 고기로45번길 40-18','경기도 용인시 수지구 고기로45번길 40-18','경기도 용인시 수지구 고기로45번길 40-18',
               '경기도 용인시 수지구 동천동 105-3','경기도 용인시 수지구 동천동 105-3','경기도 용인시 수지구 동천동 105-3','경기도 파주시 조리읍 봉일천리 13-15',
               '경기도 평택시 소사3로 53','경기도 평택시 소사3로 53','경기도 평택시 소사3로 22','경기도 평택시 소사3로 22','경기도 평택시 소사3로 22','경기도 평택시 소사3로 22',
                '경기도 평택시 용이동 601','경기도 화성시 목동 33','경기도 화성시 목동 33','경기도 화성시 목동 233','경기도 화성시 목동 206-2','경기도 화성시 목동 206-2',
                '경기도 화성시 목동 188-1','경기도 화성시 목동 188-1','경기도 화성시 목동 216-5','경기도 화성시 목동 216-5','경기도 화성시 목동 216-5',
                '경기도 화성시 새솔동 23','경기도 화성시 새솔동 23','경기도 화성시 수노을중앙로 293','경기도 화성시 새솔동 26','경기도 화성시 새솔동 18','경기도 화성시 새솔동 14',
                '경기도 화성시 영천동 503','경기도 화성시 영천동 503','경기도 화성시 영천동 651-1372','경기도 화성시 영천동 651-1372','경기도 화성시 영천동 651-1372',
                '경기도 화성시 영천동 37-16','경기도 화성시 영천동 37-16','서울특별시 강남구 역삼동 826-37','서울특별시 강남구 자곡동 686','서울특별시 서초구 내곡동 384','서울특별시 서초구 신원동 579',
                '서울특별시 서초구 신원동 579','서울특별시 서초구 신원동 579','서울특별시 성동구 금호동3가 1331','인천광역시 미추홀구 낙섬동로 135','인천광역시 미추홀구 낙섬동로 135','인천광역시 미추홀구 낙섬동로 135',
                '인천광역시 미추홀구 낙섬동로 135','인천광역시 서구 청라동 173-1','청라라임로 131','청라라임로 131','청라라임로 131','청라라임로 131',
                '인천광역시 연수구 능허대로446번길 12','인천광역시 연수구 능허대로 343','인천광역시 연수구 능허대로 343','인천광역시 연수구 능허대로 343','인천광역시 연수구 능허대로 343',
                '인천광역시 연수구 앵고개로104번길 22','인천광역시 연수구 앵고개로104번길 22','인천광역시 연수구 앵고개로104번길 22','인천광역시 연수구 봉재산로 20',
                '인천광역시 연수구 봉재산로 20','인천광역시 연수구 봉재산로 76']

In [27]:
df_temp['지번주소'] = list_address

In [32]:
# 77개의 아이디에 대해 다시 API 적용
client_id = 'fsid7bs1f2'
client_pw = 'RXQI9DaURusBXb10v6zgkeifAJXl6t2iOTxjSGaR'

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

lat = []  #위도
lng = []  #경도
for add in df_temp['지번주소']:
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except:
        pass
    try:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body:
                lat.append(response_body['addresses'][0]['y'])
                lng.append(response_body['addresses'][0]['x'])
                print('Success!')
            else:
                lat.append('')
                lng.append('')
    except:
        print('Response error code : %d' % rescode)
        lat.append('')
        lng.append('')
df_location26 = pd.DataFrame({'id':df_temp['id'],'위도':lat, '경도':lng})

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!


In [37]:
df_location.loc[df_location[df_location['위도']==''].index,['위도','경도']] = df_location26[['위도','경도']]

In [ ]:
df_location.loc[df_location['경도']=='','경도'] = df_location26['경도']

# 데이터 저장

In [41]:
df_location.to_csv('df_위도경도_11.8.csv',index=False,encoding='utf-8-sig')